# Trabalho 3, Grupo 7

### Daniel Martins a73175
### Leandro Gonçalves a74564

# Parte 1
## Codificação do autómato híbrido de um sistema ABS



Pretende-se com este exercício codificar e analisar um exemplo standard de um autómatato híbrido pelo uso do SMT Z3. O exemplo escolhido é um sistema de tavagem ABS (Anti-Blocking System). A estratégia usada prende-se com a codificação do autómato híbrido num FOTS (first order transition system), uma estrutura já trabalhada em aulas anteriores, e posterior análise dos seus traços finitos. 


In [222]:
import sys, os

from z3 import *

Comecemos por declarar as entidades involvidas na modelação. Relembra-se que um autómato híbrido é constituido por um conjunto de modos e eventos. No caso do termostáto, temos que considerar 

* os modos são $\{Start,\ Free,\ Stopping,\ Blocked,\ Stopped\}$
    - $Start$ é o modo que dá inicio ao nosso sistema;  
    - $Free$ é o modo em que o roda do carro é não está a ser travada, podendo rodar à vontade;
    - $Stopping$ é o modo em que a roda está a ser travada, diminuindo a velocidade da roda;
    - $Blocked$ é o modo em que a roda bloqueou, isto é, a roda tem velocidad 0 mas o carro continua a andar, isto significa que a roda está a deslizar,
    - $Stoppes$ é o modo em que tanto o carro como a roda estão parados, i.e., tem velocidade 0.
 
* os eventos são $\{inciar,\ alive,\ goStopping,\ goBlocked,\ goStopped,\ goFree\}$.
  
  
Autómato do sistema de ABS
![Autómato do sistema de ABS](ABS.svg)
 

In [223]:

Modes, (Start, Free, Stopping, Blocked, Stopped) = EnumSort('Modes',['Start', 'Free','Stopping', 'Blocked', 'Stopped'])
Events, (iniciar, alive, goStopping , goBlocked , goStopped , goFree) =  \
    EnumSort('Events',['iniciar', 'alive','goStopping', 'goBlocked', 'goStopped', 'goFree'])
        

Considerem-se agora as variáveis necessárias para análise de evoluções do sistema.

In [224]:
Ma = 1000    # Massa do veiculo
ma = 70     # Massa da roda
ca_estatico = 10 # Coeficiente de atrito estatico
ca_cinetico = 15 # Coeficiente de atrico cinetico

N=30
Vv   = RealVector('Vv', N+1) # Velocidade do veiculo
Vp   = RealVector('Vp', N+1) # Velocidade linear da roda
time   = RealVector('t', N+1)
modes  = AstVector() ; modes.resize(N+1)
events = AstVector() ; events.resize(N+1)

In [225]:
for k in range(N+1):
    modes[k] = Const('loc_'+str(k), Modes)
#

for k in range(N+1):
    events[k] = Const('ev_'+str(k), Events)

Teremos agora que definir uma função que nos descreva o estado do autómato em cada momento de evolução.
Cada estado será caracterizado por uma Velocidade do veiculo, uma Velocidade linear dos Peneys, um tempo e um modo corrente:

In [226]:

def S(k):
    return (Vv[k],Vp[k], time[k],modes[k])

def E(k):
    return events[k]

Definição do estado inicial:

In [227]:

def init(S):
    (Vv,Vp,T,M) = S 
    return  And( Vv == 20, Vp == 20, T == 0, M == Start)

Invariantes de permanência de estado:


In [228]:
def inv(S):
    (Vv, Vp ,T,M) = S 
    return Vv >= Vp

Dinâmica contínua do sistema:

In [229]:

def timed(S,S_,E):
    (Vv, Vp,T,M) = S ; (Vv_, Vp_,T_,M_) = S_ 
    return And( T_-T == 1, M_ == M, E == alive,
                Implies( M == Start, False),
               
                Implies( M == Free, And( Vv >= 0 , Vp >= 0,
                                         Ma*(Vv_ - Vv) == -(0.1)*(Vv - Vp), 
                                         ma*(Vp_ - Vp) == -(0.1)*(Vv - Vp) - ca_cinetico * Vp )),
                        
                Implies( M == Stopping, And( Vv >= 0, Vp >= 0,
                                             Ma*(Vv_ - Vv) == -(10)*(Vv - Vp), 
                                             ma*(Vp_ - Vp) == -(10)*(Vv - Vp) - ca_cinetico * Vp )),
                        
                Implies( M == Blocked, And( Vv != Vp,
                                            (Ma+ma)*(Vv_ - Vv) == -(ca_estatico)*(Vp_ - Vp), 
                                             Vv_ == Vp_ )),
                Implies( M == Stopped, And( Vp == Vp_, Vv == Vv_))
           )

Saltos discretos:

In [230]:
def untimed(S,S_,E): 
    (Vv, Vp ,T,M) = S ; (Vv_,Vp_,T_,M_) = S_ 
    return And( T_ == T ,
                Implies(And(M == Start, Vv > 0, Vp > 0) , And( M_ == Free, E == iniciar, Vp == Vp_, Vv == Vv_)),
                Implies(And(M == Start, Vv <= 0, Vp <= 0), False),
                Implies(And(M == Free , Vv > 0, Vp > 0) , And( M_ == Stopping, E == goStopping, Vp == Vp_, Vv == Vv_) ), 
                Implies(And(M == Free, Or(Vv <= 0, Vp <= 0) ), False),
                Implies(And(M == Stopping, Vv <= 0, Vp <= 0), And( M_ == Stopped , E == goStopped, Vp_ == 0, Vv_ == 0) ), 
                Implies(And(M == Stopping, Vv > 0 , Vp <= 0), And( M_ == Blocked , E == goBlocked, Vp_ == 0, Vv == Vv_) ), 
                Implies(And(M == Stopping, And(Vv >0, Vp > 0)), False),
                Implies(And(M == Blocked, Vv <= 0, Vp <= 0) , And( M_ == Stopped , E == goStopped, Vp_ == 0, Vv_ == 0) ), 
                Implies(And(M == Blocked, Vv > 0,  Vp == Vv), And( M_ == Free    , E == goFree, Vp == Vp_, Vv == Vv_)),
                Implies(And(M == Blocked, And(Vv > 0, Vp != Vv) ), False)
                )

Bloqueamento do sistema

In [231]:
#def blocked(S,S_,E): 
#    (X,T,M) = S ; (X_,T_,M_) = S_ 
#   return  And(E == , X_ == X, T_ == T, M_ == M)

A lei para a evolução global do sistema é assim dada por 

In [232]:
def trans(S,S_,E): 
    return  Or(timed(S,S_,E), 
               untimed(S,S_,E))

Vemos agora a consistência da especificação, i.e. a existência de evoluções no autómato modelado que preservem o invariante definido. Assim cria-se um SMT para descrever traços do sistema, iniciados no estado inicial:

In [233]:
 

def solve_s():
    s = Solver()
    set_option(rational_to_decimal=True)
    s.add(init(S(0)))

    for k in range(N):
        s.add(inv(S(k)))
        s.add(trans(S(k),S(k+1),E(k)))  

    s.add(inv(S(N)))
    return s
s = solve_s()
print(s.check())
if s.check() != unsat:
    pass
    #print(s.model())

sat


O modelo:

In [234]:
def print_model(s):
    try:
        M = s.model()
       # print(s.model())
        for k in range(N-1):
            print(M[Vv[k]], M[Vp[k]], M[time[k]], M[modes[k]])
            print("--");
            print(M[events[k]])
            print("-->")
    except Z3Exception as ex:
        print("failed: %s" % ex)
        
print_model(s)

20 20 0 Start
--
iniciar
-->
20 20 0 Free
--
goStopping
-->
20 20 0 Stopping
--
alive
-->
20 15.7142857142? 1 Stopping
--
alive
-->
19.9571428571? 11.7346938775? 2 Stopping
--
alive
-->
19.8749183673? 8.0454810495? 3 Stopping
--
alive
-->
19.7566239941? 4.6315297792? 4 Stopping
--
alive
-->
19.6053730520? 1.4783313672? 5 Stopping
--
alive
-->
19.4241026351? -1.4280313092? 6 Stopping
--
goBlocked
-->
19.4241026351? 0 6 Blocked
--
alive
-->
19.2442498329? 19.2442498329? 7 Blocked
--
goFree
-->
19.2442498329? 19.2442498329? 7 Free
--
alive
-->
19.2442498329? 15.1204820116? 8 Free
--
alive
-->
19.2438374562? 11.8744876265? 9 Free
--
goStopping
-->
19.2438374562? 11.8744876265? 9 Stopping
--
alive
-->
19.1701439579? 8.2771903023? 10 Stopping
--
alive
-->
19.0612144213? 4.9473704296? 11 Stopping
--
alive
-->
18.9200759814? 1.8709561958? 12 Stopping
--
alive
-->
18.7495847835? -0.9655515297? 13 Stopping
--
goBlocked
-->
18.7495847835? 0 13 Blocked
--
alive
-->
18.5759775170? 18.5759775170? 14

## Verificação de propriedades  
